In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time

In [103]:
driver = webdriver.Chrome()

### АВТОРИЗАЦИЯ

In [104]:
# Авторизация
username = 'odintsovmaxim4@gmail.com'
password = 'password1923DF!'

driver.get("https://www.instagram.com/")
driver.find_element(By.NAME, 'username').send_keys(username)
driver.find_element(By.NAME, 'password').send_keys(password)
driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div[2]/div/div/div[1]/div[1]/div/section/main/article/div[2]/div[1]/div[2]/div/form/div/div[3]/button/div').click()

time.sleep(5)

### Парсинг ссылок на посты

In [ ]:
# Переход на страницу с постами
searh_req = '%23rapmusic' # %23r = #
driver.get(f'https://www.instagram.com/explore/search/keyword/?q={searh_req}')

In [66]:
def get_links(parent_element) -> set:
    """Поиск ссылок"""
    link_elements = parent_element.find_elements(By.TAG_NAME, 'a')
    
    # Сбор и вывод всех ссылок
    link_list = []
    for link_element in link_elements:
        href = link_element.get_attribute("href")
        if href:
            link_list.append(href)
            
    return link_list

# Функция для скроллинга страницы и получения ссылок
def get_all_links(driver, wait_time: int = 5, max_scrolls: int = 10) -> set:
    """Скроллинг страниц и поиск ссылок"""

    last_height = driver.execute_script("return document.body.scrollHeight")
    all_links = set()
    for i in range(1, max_scrolls+1):
        # Получение ссылок
        if i % 2 == 0:
            posts_parent = driver.find_element(By.XPATH, '//*[@id="mount_0_0_lp"]/div/div/div[2]/div/div/div[1]/div[1]/div[1]/section/main/div/div[2]')
            new_links = get_links(posts_parent)
            all_links.update(new_links)

        # Скролл страницы
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(wait_time)
        new_height = driver.execute_script("return document.body.scrollHeight")
        
        if new_height == last_height:  # Если конец страницы
            break
        last_height = new_height
        
    return all_links

In [ ]:
post_links = get_all_links(driver, max_scrolls=6)

In [99]:
post_links = list(post_links)
print(f"Найдено {len(post_links)} ссылок на посты")
for href in post_links:
    print(href)

Найдено 147 ссылок на посты
https://www.instagram.com/p/DC7f5nUT88d/
https://www.instagram.com/p/C7z1CD-tCML/
https://www.instagram.com/p/C8xFj9csmyS/
https://www.instagram.com/p/C2P1PStrSuv/
https://www.instagram.com/p/DBW2E_fuM1c/
https://www.instagram.com/p/C634VchLQTK/
https://www.instagram.com/p/DC5dVortVoD/
https://www.instagram.com/p/C9iDI6epZ1T/
https://www.instagram.com/p/C-tCyIjvgIQ/
https://www.instagram.com/p/DCUVdnHzbZy/
https://www.instagram.com/p/DBldU4FyT6K/
https://www.instagram.com/p/DBKpTjFCJ6d/
https://www.instagram.com/p/DCIxVtVSQ7q/
https://www.instagram.com/p/C1_s9LvLNhJ/
https://www.instagram.com/p/C2xeghhvapc/
https://www.instagram.com/p/C8N5z_wvYIu/
https://www.instagram.com/p/DA1_5zHsw8e/
https://www.instagram.com/p/C69PW9zSeqW/
https://www.instagram.com/p/C4FuCd0JKPz/
https://www.instagram.com/p/DDORqIMTDMV/
https://www.instagram.com/p/C6ySIInBtgK/
https://www.instagram.com/p/C2pn7LbILCj/
https://www.instagram.com/p/C3Y-4khIF_F/
https://www.instagram.com/p/C

### Парсинг ссылок на аккаунты

In [121]:
account_links = set()
for post_link in post_links[:30]:
    # driver.switch_to.new_window('tab')
    driver.get(post_link)
    time.sleep(2)
    try:
        # Первичный XPath
        accounts_parent_xpath = '/html/body/div[2]/div/div/div[2]/div/div/div[1]/div[1]/div[1]/section/main/div/div[1]/div/div[2]/div/div[1]/div/div[2]/div/div[1]/div[1]/div/div/div/span/span'
        accounts_parent = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.XPATH, accounts_parent_xpath))
        )
    except Exception:
        # Альтернативный XPath
        alt_accounts_parent_xpath = '/html/body/div[2]/div/div/div[2]/div/div/div[1]/div[1]/div[1]/section/main/div/div[1]/div/div[2]/div/div[1]/div/div[2]/div/div[1]/div/div/div'
        print(f'Не найден элемент {accounts_parent_xpath}. Ищу {alt_accounts_parent_xpath}')
        
        try:
            accounts_parent = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.XPATH, alt_accounts_parent_xpath))
            )
        except Exception as e:
            print(f"Ошибка: {e}")
            continue  # Пропустить итерацию, если элемент не найден
        
    new_links = get_links(accounts_parent)
    print(f'Найдены ссылки: {new_links}')
    account_links.update(new_links)
    
    # driver.close()
# driver.execute_script("window.open('https://www.google.com', '_blank');")

Найдены ссылки: ['https://www.instagram.com/rak.ib940/']
Найдены ссылки: ['https://www.instagram.com/steph_anie_green/']
Найдены ссылки: ['https://www.instagram.com/rapgoodent/']
Найдены ссылки: ['https://www.instagram.com/rapmusic/']
Найдены ссылки: ['https://www.instagram.com/tracks/']
Найдены ссылки: ['https://www.instagram.com/cali.bratt/']
Найдены ссылки: ['https://www.instagram.com/atlantadisccd/']
Найдены ссылки: ['https://www.instagram.com/yoshi_yin/']
Найдены ссылки: ['https://www.instagram.com/290zipp/']
Найдены ссылки: ['https://www.instagram.com/prishaaaa__/']
Найдены ссылки: ['https://www.instagram.com/saycheesedigitall/']
Найдены ссылки: ['https://www.instagram.com/jsam1805/']
Не найден элемент /html/body/div[2]/div/div/div[2]/div/div/div[1]/div[1]/div[1]/section/main/div/div[1]/div/div[2]/div/div[1]/div/div[2]/div/div[1]/div[1]/div/div/div/span/span. Ищу /html/body/div[2]/div/div/div[2]/div/div/div[1]/div[1]/div[1]/section/main/div/div[1]/div/div[2]/div/div[1]/div/div[2]

In [122]:
account_links = list(account_links)
print(f"Найдено {len(account_links)} ссылок на аккаунты")
for href in account_links:
    print(href)

Найдено 28 ссылок на аккаунты
https://www.instagram.com/rap.uptv/
https://www.instagram.com/rak.ib940/
https://www.instagram.com/hiphopdiarie/
https://www.instagram.com/rapinsoul/
https://www.instagram.com/atlantadisccd/
https://www.instagram.com/tracks/
https://www.instagram.com/prishaaaa__/
https://www.instagram.com/dc_therapperx/
https://www.instagram.com/blizzynokizzy/
https://www.instagram.com/music___records__/
https://www.instagram.com/__real_promotion/
https://www.instagram.com/grow_artist01/
https://www.instagram.com/steph_anie_green/
https://www.instagram.com/yoshi_yin/
https://www.instagram.com/rapmusic/
https://www.instagram.com/jsam1805/
https://www.instagram.com/dc_therapper/
https://www.instagram.com/zee07___/
https://www.instagram.com/worldstar__talent__ig__/
https://www.instagram.com/digginconnection/
https://www.instagram.com/saycheesedigitall/
https://www.instagram.com/cali.bratt/
https://www.instagram.com/p.henny1/
https://www.instagram.com/a1makeyoupay1/
https://ww

### Парсинг страничек

In [142]:
from urllib.parse import urlparse, parse_qs


def extract_original_link(instagram_link):
    """
    Функция извлекает оригинальную ссылку
    """
    parsed_url = urlparse(instagram_link)
    query_params = parse_qs(parsed_url.query)
    return query_params.get("u", [""])[0]


def trim_link(link: str) -> str:
    """
    Функция обрезает ссылку, оставляя только основную часть до символа `?`.
    """
    if '?' in link:
        return link.split('?')[0]
    else:
        return link




In [124]:
account_link = account_links[0]
account_link

'https://www.instagram.com/rap.uptv/'

In [126]:
link = 'https://www.instagram.com/itsfiftykbaby'
driver.get(link)

In [127]:
profession_el = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div[2]/div/div/div[1]/div[2]/div/div[1]/section/main/div/header/section[4]/div/div[3]/div')
profession_el.text

'Деятель искусств'

In [ ]:
# Если одна ссылка
links_el = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div[2]/div/div/div[1]/div[2]/div/div[1]/section/main/div/header/section[4]/div/div[4]/div/a/span/span')
links_el.text

'youtu.be/0gqF7vszHyg?si=eDvTrky4cVhf95ht'

In [149]:
# Ссылка + n ссылок
links_el = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div[2]/div/div/div[1]/div[2]/div/div[1]/section/main/div/header/section[4]/div/button/div')
links_el.click()

In [147]:
links_modal_parent_el = driver.find_element(By.XPATH, "/html/body/div[5]/div[1]/div/div[2]/div/div/div/div/div/div/div/div")
links = get_links(links_modal_parent_el)
original_links = [trim_link(extract_original_link(link)) for link in links]
print(original_links)

['https://song.link/chaseadime', 'https://album.link/zhbfrj5wcmbvd']


In [151]:
accounts_info = []
for account_link in account_links:
    driver.get(account_link)
    time.sleep(3)
    
    links = None
    profession = None
    
    #####################
    # Парсинг профессии #
    #####################
    try:
        profession_el = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div[2]/div/div/div[1]/div[2]/div/div[1]/section/main/div/header/section[4]/div/div[3]/div')
        profession = profession_el.text
    except:
        pass
    
    ##################
    # Парсинг ссылок #
    ##################
    try:
        # Парсинг, если одна ссылка
        links_xpath = '/html/body/div[2]/div/div/div[2]/div/div/div[1]/div[2]/div/div[1]/section/main/div/header/section[4]/div/div[4]/div/a/span/span'
        links_el = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, links_xpath)))
        links = [links_el.text]
    except Exception:
        modal_links_el_xpath = '/html/body/div[2]/div/div/div[2]/div/div/div[1]/div[2]/div/div[1]/section/main/div/header/section[4]/div/button/div'
        links_parent_el_xpath = '/html/body/div[5]/div[1]/div/div[2]/div/div/div/div/div/div/div/div'        
        try:
            # Открытие модального окна
            modal_links_el = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, modal_links_el_xpath)))
            # modal_links_el = driver.find_element(By.XPATH, modal_links_el_xpath)
            modal_links_el.click()
            
            # Парсинг ссылок в модальном окне
            links_el = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, links_parent_el_xpath)))
            links_modal_parent_el = driver.find_element(By.XPATH, links_parent_el_xpath)
            find_links = get_links(links_modal_parent_el)
            links = [trim_link(extract_original_link(find_link)) for find_link in find_links]
        except Exception as e:
            print(f'Ошибка: Не найден элемент ссылок. Полный лог: {e}')
            pass
    
    print(f'Ссылка на аккаунт: {account_link}\tПрофессия: {profession}\t Ссылки: {links}')
    accounts_info.append({'account_link': account_link, 'profession': profession, 'links': links})
    

Ссылка на аккаунт: https://www.instagram.com/rap.uptv/	Профессия: Музыкант/группа	 Ссылки: ['https://youtu.be/d--91IdORq4', 'https://www.instagram.com/p/DCXXNW2z-pX/']
Ошибка: Не найден элемент ссылок. Полный лог: Message: 
Stacktrace:
#0 0x5e697a11d34a <unknown>
#1 0x5e6979c336e0 <unknown>
#2 0x5e6979c823e6 <unknown>
#3 0x5e6979c82681 <unknown>
#4 0x5e6979cc7b04 <unknown>
#5 0x5e6979ca648d <unknown>
#6 0x5e6979cc4ed7 <unknown>
#7 0x5e6979ca6203 <unknown>
#8 0x5e6979c74cc0 <unknown>
#9 0x5e6979c75c9e <unknown>
#10 0x5e697a0ead3b <unknown>
#11 0x5e697a0eecc2 <unknown>
#12 0x5e697a0d7b6c <unknown>
#13 0x5e697a0ef837 <unknown>
#14 0x5e697a0bd10f <unknown>
#15 0x5e697a10c5a8 <unknown>
#16 0x5e697a10c770 <unknown>
#17 0x5e697a11c1c6 <unknown>
#18 0x76096d094ac3 <unknown>

Ссылка на аккаунт: https://www.instagram.com/rak.ib940/	Профессия: None	 Ссылки: None
Ссылка на аккаунт: https://www.instagram.com/hiphopdiarie/	Профессия: Маркетинговое агентство	 Ссылки: ['https://youtu.be/14qceuscFPc', 

In [152]:
accounts_info

[{'account_link': 'https://www.instagram.com/rap.uptv/',
  'profession': 'Музыкант/группа',
  'links': ['https://youtu.be/d--91IdORq4',
   'https://www.instagram.com/p/DCXXNW2z-pX/']},
 {'account_link': 'https://www.instagram.com/rak.ib940/',
  'profession': None,
  'links': None},
 {'account_link': 'https://www.instagram.com/hiphopdiarie/',
  'profession': 'Маркетинговое агентство',
  'links': ['https://youtu.be/14qceuscFPc',
   'https://youtube.com/shorts/HEy7b-4pQtA',
   'https://music.apple.com/us/artist/sl-draco/1566260863',
   'https://on.soundcloud.com/Xnj4Tteo9KyH6FLg7',
   'https://onlinepromobro.com/']},
 {'account_link': 'https://www.instagram.com/rapinsoul/',
  'profession': 'СМИ/информационное агентство',
  'links': None},
 {'account_link': 'https://www.instagram.com/atlantadisccd/',
  'profession': 'www.atlantadisc.com',
  'links': None},
 {'account_link': 'https://www.instagram.com/tracks/',
  'profession': None,
  'links': None},
 {'account_link': 'https://www.instagram

In [155]:
import pandas as pd

In [157]:
df = pd.DataFrame.from_dict(accounts_info)
pd.set_option('display.width', None)  # Автоматическая ширина вывода
pd.set_option('display.max_columns', None)  # Показывать все столбцы

In [162]:
import json
with open('data.json', 'w') as f:
    js = json.dumps(accounts_info, indent=4)
    f.write(js)